In [8]:
# Uses code from 'make j from scratch'
D.<x> = PolynomialRing(QQ)
R.<x> = PolynomialRing(QQ)
S = PowerSeriesRing(QQ,'x', default_prec=300)
L=LaurentSeriesRing(QQ,'x', default_prec=300)
#S.<x,y>=PowerSeriesRing(QQ,default_prec=300)
# Using "R = PolynomialRing(QQ,'x')" produces an error.
# Yet using S as defined here is fine (see definition of Fraleigh4 below)
# But S = PowerSeriesRing(QQ,'x','y') also produces an error.
def pochhammer(x,n):
    return product ([x+k for k in [0..n-1]])
def cRaleigh2(alpha,beta,nu):
    pchhmrA = pochhammer(alpha,nu)
    pchhmrB = pochhammer(beta,nu)
    fnu=factorial(nu)
    answer = D(pchhmrA*pchhmrB/fnu^2)
    return answer
def term(alpha,beta,p):
    return D(1/(alpha + p) + 1/(beta + p) - 2/(1 + p))
def eRaleigh(alpha, beta, nu):
    return D(sum([term(alpha, beta, p) for p in [0..nu - 1]]))
def FstarRaleigh2(alpha,beta,u,terms):
    fsr = sum([cRaleigh2(alpha, beta, nu)*eRaleigh(alpha,beta,nu)*u^nu for nu in [1..terms]])
    return D(fsr)
def Fraleigh2(alpha,beta,u,terms): 
    return D(sum([cRaleigh2(alpha,beta,nu)*u^nu for nu in [0..terms]]))
def FstarRaleigh3(n,m,x):
    alpha = (1/2-1/m)/2
    beta = (1/2+1/m)/2
    fsr2 = FstarRaleigh2(alpha,beta,x,n)
    return D(fsr2)
def Fraleigh4(n,m,x):
    alpha = (1/2-1/m)/2
    beta = (1/2+1/m)/2
    fr2 = Fraleigh2(alpha,beta,x,n)
    return D(fr2)
def exNo3c(n,m):
    a1 = S(x)*exp(S(FstarRaleigh3(n,m,x)/Fraleigh4(n,m,x))).O(2*n)
    # S(x) instead of x in df of a1 to avoid an error message
    # after use of the stripQuotationMarks command below.
    a2 = S(a1).O(n+1)
    return a2 # applying S to argument of exp is key.
def J(n,m):
    exn = exNo3c(n+2,m)
    jnm = 1/exn.reverse()
    return jnm.O(n+1) # Jtest is faster than J, and they agree.
def normalJ(n,m):
    f = J(n,m)
    return f.truncate(n+1)
def Jcoefficient(n,m):
    f = J(n+2,m)
    f1 = f.truncate(n+1)
    f1=f1.list()
    f1=Set(f1)
    g1 = f.truncate(n)
    g1 = g1.list()
    g1 = set(g1)
    h = f1.difference(g1)
    h = h.list()
    return h[0]
def xjNew(n,m):
    def xjNewStep(n,m):
        f=J(n,m)
        fl=f.list()
        rt=0
        for i in [0..len(fl)-1]:
            rt=rt+fl[i]*x**i*(m**3*2**6)**i
        return rt
    return xjNewStep(n-1,m)
def j(n,m):return L(xjNew(n,m)/x)
def tildeHsubI(n,m):
    jay = j(n+1,m)
    numerator = x^m*derivative(jay,x)^m
    denominator = jay^(m-1)*(jay-2^6*m^3)
    power=1/(m-2)
    return L((numerator/denominator)^power).truncate(n)
def polynomialCoefficient(n,poly):
    return poly.list()[n]
def leadingCoefficient(polynomial):
    dg=polynomial.degree()
    return polynomialCoefficient(dg,polynomial)
def factorpairToFactor(factorpair):
    (monomial,exponent)=factorpair
    return monomial^exponent
def numericalfactor(polynomial):
    fp=polynomial.factor()
    lst=[factorpairToFactor(factorpair) for factorpair in fp]
    return polynomial/product(lst)
def firstfactor(polynomial):
    fp=polynomial.factor()
    fp=FiniteEnumeratedSet(fp)
    return factorpairToFactor(fp.first())
def lastfactor(polynomial):
    fp=polynomial.factor()
    fp=FiniteEnumeratedSet(fp)
    return factorpairToFactor(fp.last())
def reduced(polynomial):
    return numericalfactor(polynomial)*lastfactor(polynomial)
def stripQuotationMarks(string):
    var('x')
    return sage_eval(string,locals={'x':x})
def sigmaOdd(n):
    sum=0
    dvs=divisors(n)
    lnth=len(dvs)
    for k in [0..lnth-1]:
        dv=dvs[k]
        if mod(dv,2)==1:
            sum=sum+dv
    return sum

def division(dividend, divisor) : 
    quotient=(dividend._maxima_().divide(divisor).sage())[0] 
    remainder=(dividend._maxima_().divide(divisor).sage())[1] 
    return (quotient,remainder)

In [2]:
j(10,3)

x^-1 + 744 + 196884*x + 21493760*x^2 + 864299970*x^3 + 20245856256*x^4 + 333202640600*x^5 + 4252023300096*x^6 + 44656994071935*x^7 + 401490886656000*x^8 + 3176440229784420*x^9

In [3]:
def tildeHsubI(n,m):
    jay = j(n+1,m)
    numerator = x^m*derivative(jay,x)^m
    denominator = jay^(m-1)*(jay-2^6*m^3)
    power=1/(m-2)
    return L((numerator/denominator)^power).truncate(n)

In [4]:
n=5
for m in [3..6]:
    print tildeHsubI(n,m) # will have to normalize these to obtain "valid" interpolating polynomials.
    print

-1 + 504*x + 16632*x^2 + 122976*x^3 + 532728*x^4

1 - 1280*x - 102400*x^2 - 9175040*x^3 - 193986560*x^4

-1 + 2600*x + 351000*x^2 + 276380000/3*x^3 + 29091725000/3*x^4

1 - 4608*x - 884736*x^2 - 528482304*x^3 - 125627793408*x^4



In [5]:
n=5
for m in [3..6]:
    print tildeHsubI(n,m)*(-1)^m
    print

1 - 504*x - 16632*x^2 - 122976*x^3 - 532728*x^4

1 - 1280*x - 102400*x^2 - 9175040*x^3 - 193986560*x^4

1 - 2600*x - 351000*x^2 - 276380000/3*x^3 - 29091725000/3*x^4

1 - 4608*x - 884736*x^2 - 528482304*x^3 - 125627793408*x^4



In [3]:
import pickle
rfile = open('/Users/barrybrent/28jun21no3.txt','r') # too large for github; will break it up below,
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
len(s)

308

In [6]:
import pickle
rfile = open('/Users/barrybrent/28jun21no3.txt','r') # too large for github; will break it up below,
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)

wfile1 = open('/Users/barrybrent/run30jun21no22.txt','a') # tilde{H}_{i,m}
data1=[]
for k in [0..54]:
    print k
    data1=data1+[s[k]]
t1 = pickle.dumps(str(data1))
wfile1.write(t1)
wfile1.close()

wfile2 = open('/Users/barrybrent/run30jun21no23.txt','a') # tilde{H}_{i,m}
data2=[]
for k in [55..110]:
    print k
    data2=data2+[s[k]]
t2 = pickle.dumps(str(data2))
wfile2.write(t2)
wfile2.close()

wfile3 = open('/Users/barrybrent/run30jun21no24.txt','a') # tilde{H}_{i,m}
data3=[]
for k in [111..165]:
    print k
    data3=data3+[s[k]]
t3 = pickle.dumps(str(data3))
wfile3.write(t3)
wfile3.close()

wfile4 = open('/Users/barrybrent/run30jun21no25.txt','a') # tilde{H}_{i,m}
data4=[]
for k in [165..220]:
    print k
    data4=data4+[s[k]]
t4 = pickle.dumps(str(data4))
wfile4.write(t4)
wfile4.close()

wfile5 = open('/Users/barrybrent/run30jun21no26.txt','a') # tilde{H}_{i,m}
data5=[]
for k in [221..275]:
    print k
    data5=data5+[s[k]]
t5 = pickle.dumps(str(data5))
wfile5.write(t5)
wfile5.close()

wfile6 = open('/Users/barrybrent/run30jun21no27.txt','a') # tilde{H}_{i,m}
data6=[]
for k in [276..307]:
    print k
    data6=data6+[s[k]]
t6 = pickle.dumps(str(data6))
wfile6.write(t6)
wfile6.close()

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
27

In [6]:
import pickle
rfile = open('/Users/barrybrent/28jun21no3.txt','r')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)


wfile = open('/Users/barrybrent/run30jun21no7.txt','a') # tilde{H}_{i,m} polynomials
polydata=[]
for n in [0..99]:
    print n
    data=[]
    for k in [0..len(s)-1]:
        m = s[k][0]
        poly=s[k][1]*(-1)^m # this is the normalization
        cf=polynomialCoefficient(n,poly)
        data=data+[[m,cf]]
    rl=R.lagrange_polynomial(data)
    polydata=polydata+[[n,rl]] 

    
t = pickle.dumps(str(polydata))
wfile.write(t)
wfile.close()

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99


In [7]:
import pickle
rfile = open('/Users/barrybrent/run30jun21no7.txt','r') # tilde{H}_{i,m} polynomials
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
len(s)



100

In [37]:
import pickle
rfile = open('/Users/barrybrent/run30jun21no7.txt','r') # tilde{H}_{i,m} polynomials
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
for k in [0..6]:
    n=s[k][0]
    poly=D(s[k][1])
    print "===================================================================================="
    print "n: ",n
    print 
    print poly
    print
    print poly.factor()
    

n:  0

1

1
n:  1

-24*x^3 + 16*x^2

(-24) * (x - 2/3) * x^2
n:  2

24*x^6 - 400*x^5 + 928*x^4 - 448*x^3

(24) * (x - 14) * (x - 2) * (x - 2/3) * x^3
n:  3

-96*x^9 + 2368/3*x^8 - 58112/9*x^7 + 230912/9*x^6 - 343552/9*x^5 + 142336/9*x^4

(-96) * (x - 2) * (x - 2/3) * x^4 * (x^3 - 50/9*x^2 + 460/9*x - 1112/9)
n:  4

24*x^12 - 2448*x^11 + 211168/9*x^10 - 3498944/27*x^9 + 543872*x^8 - 37402880/27*x^7 + 14776832/9*x^6 - 16667648/27*x^5

(24) * (x - 2) * (x - 2/3) * x^5 * (x^5 - 298/3*x^4 + 19208/27*x^3 - 90992/27*x^2 + 343600/27*x - 520864/27)
n:  5

-144*x^15 + 14176/5*x^14 - 11656448/225*x^13 + 352563712/675*x^12 - 2036271616/675*x^11 + 8120388608/675*x^10 - 24314884096/675*x^9 + 48048939008/675*x^8 - 49091129344/675*x^7 + 17211891712/675*x^6

(-144) * (x - 2) * (x - 2/3) * x^6 * (x^7 - 766/45*x^6 + 633908/2025*x^5 - 5608696/2025*x^4 + 26620592/2025*x^3 - 90708256/2025*x^2 + 45835456/405*x - 268935808/2025)
n:  6

96*x^18 - 84032/15*x^17 + 38871424/375*x^16 - 4217302784/3375*x^15 + 21920

In [16]:
import pickle
rfile = open('/Users/barrybrent/run30jun21no7.txt','r') # tilde{H}_{i,m} polynomials
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
for k in [0..6]:
    n=s[k][0]
    poly=D(s[k][1])
    print "===================================================================================="
    print "n: ",n
    print 
    print poly.factor()
    print numericalfactor(poly)

n:  0

1
1
n:  1

(-24) * (x - 2/3) * x^2
-24
n:  2

(24) * (x - 14) * (x - 2) * (x - 2/3) * x^3
24
n:  3

(-96) * (x - 2) * (x - 2/3) * x^4 * (x^3 - 50/9*x^2 + 460/9*x - 1112/9)
-96
n:  4

(24) * (x - 2) * (x - 2/3) * x^5 * (x^5 - 298/3*x^4 + 19208/27*x^3 - 90992/27*x^2 + 343600/27*x - 520864/27)
24
n:  5

(-144) * (x - 2) * (x - 2/3) * x^6 * (x^7 - 766/45*x^6 + 633908/2025*x^5 - 5608696/2025*x^4 + 26620592/2025*x^3 - 90708256/2025*x^2 + 45835456/405*x - 268935808/2025)
-144
n:  6

(96) * (x - 2) * (x - 2/3) * x^7 * (x^9 - 2506/45*x^8 + 3138496/3375*x^7 - 105930944/10125*x^6 + 846635552/10125*x^5 - 4558010176/10125*x^4 + 5527929344/3375*x^3 - 44408969216/10125*x^2 + 87306627328/10125*x - 86403019264/10125)
96


In [11]:
import pickle
rfile = open('/Users/barrybrent/run30jun21no7.txt','r') # tilde{H}_{i,m} polynomials
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
data=[]
for k in [0..len(s)-1]:
    n=s[k][0]
    poly=D(s[k][1])
    nf=numericalfactor(poly)
    data=data+[nf]
print data # data doesn't match the above factorizations;
# maybe it matches when the factorization is into monics.

[1, -24, 24, -96, 24, -144, 96, -192, 24, -312, 144, -288, 96, -336, 192, -576, 24, -432, 312, -480, 144, -768, 288, -576, 96, -744, 336, -960, 192, -720, 576, -768, 24, -1152, 432, -1152, 312, -912, 480, -1344, 144, -1008, 768, -1056, 288, -1872, 576, -1152, 96, -1368, 744, -1728, 336, -1296, 960, -1728, 192, -1920, 720, -1440, 576, -1488, 768, -2496, 24, -2016, 1152, -1632, 432, -2304, 1152, -1728, 312, -1776, 912, -2976, 480, -2304, 1344, -1920, 144, -2904, 1008, -2016, 768, -2592, 1056, -2880, 288, -2160, 1872, -2688, 576, -3072, 1152, -2880, 96, -2352, 1368, -3744]


In [12]:
import pickle
rfile = open('/Users/barrybrent/run30jun21no7.txt','r') # tilde{H}_{i,m} polynomials
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
data=[]
for k in [1..len(s)-1]:
    n=s[k][0]
    poly=D(s[k][1])
    nf=numericalfactor(poly)
    data=data+[nf/24]
print data

[-1, 1, -4, 1, -6, 4, -8, 1, -13, 6, -12, 4, -14, 8, -24, 1, -18, 13, -20, 6, -32, 12, -24, 4, -31, 14, -40, 8, -30, 24, -32, 1, -48, 18, -48, 13, -38, 20, -56, 6, -42, 32, -44, 12, -78, 24, -48, 4, -57, 31, -72, 14, -54, 40, -72, 8, -80, 30, -60, 24, -62, 32, -104, 1, -84, 48, -68, 18, -96, 48, -72, 13, -74, 38, -124, 20, -96, 56, -80, 6, -121, 42, -84, 32, -108, 44, -120, 12, -90, 78, -112, 24, -128, 48, -120, 4, -98, 57, -156]


In [ ]:
# sigmaOdd(n)*(-1)^n  OEIS A000593 auth. NJA Sloane URL = http://oeis.org/A000593

In [31]:
import pickle
rfile = open('/Users/barrybrent/run30jun21no7.txt','r') # tilde{H}_{i,m} polynomials
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
data=[]
for k in [1..len(s)-1]:
    n=s[k][0]
    poly=D(s[k][1])
    nf=numericalfactor(poly)
    data=data+[nf/24-sigmaOdd(n)*(-1)^n]
print data

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [34]:
f=x^2;f.degree(x)

2

In [36]:
import pickle
rfile = open('/Users/barrybrent/run30jun21no7.txt','r') # tilde{H}_{i,m} polynomials
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
data=[]
for k in [1..len(s)-1]:
    n=s[k][0]
    poly=D(s[k][1])
    dg=poly.degree() # under D coercion, degree.() takes no argument.
    data=data+[3*n-dg]
print data

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [9]:
import pickle
rfile = open('/Users/barrybrent/run30jun21no7.txt','r') # tilde{H}_{i,m} polynomials
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
for k in [0..6]:
    n=s[k][0]
    poly=s[k][1]
    print "===================================================================================="
    print "n: ",n
    print
    print n
    print division(poly,x-2)
   

n:  0

0
(0, 1)
n:  1

1
(-24*x^2 - 32*x - 64, -128)
n:  2

2
(24*x^5 - 352*x^4 + 224*x^3, 0)
n:  3

3
(-96*x^8 + 1792/3*x^7 - 47360/9*x^6 + 136192/9*x^5 - 71168/9*x^4, 0)
n:  4

4
(24*x^11 - 2400*x^10 + 167968/9*x^9 - 2491136/27*x^8 + 9702272/27*x^7 - 17998336/27*x^6 + 8333824/27*x^5, 0)
n:  5

5
(-144*x^14 + 12736/5*x^13 - 10510208/225*x^12 + 289502464/675*x^11 - 1457266688/675*x^10 + 5205855232/675*x^9 - 13903173632/675*x^8 + 20242591744/675*x^7 - 8605945856/675*x^6, 0)
n:  6

6
(96*x^17 - 81152/15*x^16 + 11604608/125*x^15 - 3590653952/3375*x^14 + 88056593408/10125*x^13 - 491753652224/10125*x^12 + 1883756302336/10125*x^11 - 197208276992/375*x^10 + 11223610253312/10125*x^9 - 13882313998336/10125*x^8 + 5529793232896/10125*x^7, 0)


In [11]:
import pickle
rfile = open('/Users/barrybrent/run30jun21no7.txt','r') # tilde{H}_{i,m} polynomials
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
remainders=[]
for k in [0..len(s)-1]:
    n=s[k][0]
    poly=s[k][1]
    dv=division(poly,x-2)
    remainders= remainders+[dv[1]]
print remainders
   

[1, -128, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [12]:
import pickle
rfile = open('/Users/barrybrent/run30jun21no7.txt','r') # tilde{H}_{i,m} polynomials
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
remainders=[]
for k in [0..len(s)-1]:
    n=s[k][0]
    poly=s[k][1]
    dv=division(poly,x-2/3)
    remainders= remainders+[dv[1]]
print remainders

[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
